# 6-1. 프로젝트 : 네이버 영화리뷰 감성분석 도전하기

데이터 : 네이버 영화의 댓글을 모아 구성된 [Naver sentiment movie corpus](https://github.com/e9t/nsmc)


평가문항	상세기준
1. 다양한 방법으로 Text Classification 태스크를 성공적으로 구현하였다.	3가지 이상의 모델이 성공적으로 시도됨
2. gensim을 활용하여 자체학습된 혹은 사전학습된 임베딩 레이어를 분석하였다.	gensim의 유사단어 찾기를 활용하여 자체학습한 임베딩과 사전학습 임베딩을 비교 분석함
3. 한국어 Word2Vec을 활용하여 가시적인 성능향상을 달성했다.	네이버 영화리뷰 데이터 감성분석 정확도를 85% 이상 달성함

In [55]:
import pandas as pd
import konlpy
import gensim
from konlpy.tag import Mecab
import numpy as np
from collections import Counter
import tensorflow as tf


# 데이터를 읽어봅시다. 
train_data = pd.read_table('./data/ratings_train.txt')
test_data = pd.read_table('./data/ratings_test.txt')

train_data.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [56]:
# 문장 1개를 활용할 딕셔너리와 함께 주면, 단어 인덱스 리스트 벡터로 변환해 주는 함수입니다. 
# 단, 모든 문장은 <BOS>로 시작하는 것으로 합니다. 
def get_encoded_sentence(sentence, word_to_index):
    return [word_to_index['<BOS>']]+[word_to_index[word] if word in word_to_index else word_to_index['<UNK>'] for word in sentence.split()]

# 여러 개의 문장 리스트를 한꺼번에 단어 인덱스 리스트 벡터로 encode해 주는 함수입니다. 
def get_encoded_sentences(sentences, word_to_index):
    return [get_encoded_sentence(sentence, word_to_index) for sentence in sentences]

# 숫자 벡터로 encode된 문장을 원래대로 decode하는 함수입니다. 
def get_decoded_sentence(encoded_sentence, index_to_word):
    return ' '.join(index_to_word[index] if index in index_to_word else '<UNK>' for index in encoded_sentence[1:])  #[1:]를 통해 <BOS>를 제외

# 여러 개의 숫자 벡터로 encode된 문장을 한꺼번에 원래대로 decode하는 함수입니다. 
def get_decoded_sentences(encoded_sentences, index_to_word):
    return [get_decoded_sentence(encoded_sentence, index_to_word) for encoded_sentence in encoded_sentences]

In [57]:
# 데이터의 중복 제거
# NaN 결측치 제거
# 한국어 토크나이저로 토큰화
# 불용어(Stopwords) 제거
# 사전word_to_index 구성
# 텍스트 스트링을 사전 인덱스 스트링으로 변환
# X_train, y_train, X_test, y_test, word_to_index 리턴

tokenizer = Mecab()
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

def load_data(train_data, test_data, num_words=10000):
    train_data.drop_duplicates(subset=['document'], inplace=True)
    train_data = train_data.dropna(how = 'any') 
    test_data.drop_duplicates(subset=['document'], inplace=True)
    test_data = test_data.dropna(how = 'any') 
    
    X_train = []
    for sentence in train_data['document']:
        temp_X = tokenizer.morphs(sentence) # 토큰화
        temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
        X_train.append(temp_X)

    X_test = []
    for sentence in test_data['document']:
        temp_X = tokenizer.morphs(sentence) # 토큰화
        temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
        X_test.append(temp_X)
    
    words = np.concatenate(X_train).tolist()
    counter = Counter(words)
    counter = counter.most_common(10000-4)
    vocab = ['', '', '', ''] + [key for key, _ in counter]
    word_to_index = {word:index for index, word in enumerate(vocab)}
        
    def wordlist_to_indexlist(wordlist):
        return [word_to_index[word] if word in word_to_index else word_to_index[''] for word in wordlist]
        
    X_train = list(map(wordlist_to_indexlist, X_train))
    X_test = list(map(wordlist_to_indexlist, X_test))
        
    return X_train, np.array(list(train_data['label'])), X_test, np.array(list(test_data['label'])), word_to_index
    
X_train, y_train, X_test, y_test, word_to_index = load_data(train_data, test_data) 

In [58]:
print(X_train[0])  # 1번째 리뷰데이터
print('라벨: ', y_train[0])  # 1번째 리뷰데이터의 라벨
print('1번째 리뷰 문장 길이: ', len(X_train[0]))
print('2번째 리뷰 문장 길이: ', len(X_train[1]))

[32, 74, 919, 4, 4, 39, 228, 20, 33, 748]
라벨:  0
1번째 리뷰 문장 길이:  10
2번째 리뷰 문장 길이:  17


In [59]:
index_to_word = {index:word for word, index in word_to_index.items()}
print(get_decoded_sentence(X_train[0], index_to_word))

더 빙 . . 진짜 짜증 나 네요 목소리


In [60]:
len(X_train), len(y_train), len(X_test), len(y_test), len(word_to_index)

(146182, 146182, 49157, 49157, 9997)

In [106]:
# 값으로 키 찾기
index = 3
word = list(word_to_index.keys())[list(word_to_index.values()).index(index)]
print(word) # 출력: ''

0,1,2 에 매칭된 데이터 없음

In [64]:
word_to_index["<PAD>"] = 0
word_to_index["<BOS>"] = 1
word_to_index["<UNK>"] = 2  # unknown

index_to_word = {index:word for word, index in word_to_index.items()}

In [66]:
print(index_to_word[1])     # '<BOS>' 가 출력됩니다. 
print(word_to_index['the'])  
print(index_to_word[1909])   

# 보정 후 x_train[0] 데이터
print(get_decoded_sentence(X_train[0], index_to_word))
print('라벨: ', y_train[0])  # 1번째 리뷰데이터의 라벨

<BOS>
1909
the
더 빙 . . 진짜 짜증 나 네요 목소리
라벨:  0


In [67]:
total_data_text = list(X_train) + list(X_test)
# 텍스트데이터 문장길이의 리스트를 생성한 후
num_tokens = [len(tokens) for tokens in total_data_text]
num_tokens = np.array(num_tokens)
# 문장길이의 평균값, 최대값, 표준편차를 계산해 본다. 
print('문장길이 평균 : ', np.mean(num_tokens))
print('문장길이 최대 : ', np.max(num_tokens))
print('문장길이 표준편차 : ', np.std(num_tokens))

# 예를들어, 최대 길이를 (평균 + 2*표준편차)로 한다면,  
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
maxlen = int(max_tokens)
print('pad_sequences maxlen : ', maxlen)
print(f'전체 문장의 {np.sum(num_tokens < max_tokens) / len(num_tokens)}%가 maxlen 설정값 이내에 포함됩니다. ')

문장길이 평균 :  15.969376315021577
문장길이 최대 :  116
문장길이 표준편차 :  12.843535456326455
pad_sequences maxlen :  41
전체 문장의 0.9342988343341575%가 maxlen 설정값 이내에 포함됩니다. 


In [69]:
x_train = tf.keras.preprocessing.sequence.pad_sequences(X_train,
                                                        value=word_to_index["<PAD>"],
                                                        padding='pre',
                                                        maxlen=maxlen)

x_test = tf.keras.preprocessing.sequence.pad_sequences(X_test,
                                                       value=word_to_index["<PAD>"],
                                                       padding='pre', 
                                                       maxlen=maxlen)

print(x_train.shape)

(146182, 41)


In [81]:
train_val_split = int(0.2*x_train.shape[0])

# validation set 
x_val = x_train[:train_val_split]   
y_val = y_train[:train_val_split]

# validation set을 제외한 나머지 15000건
partial_x_train = x_train[train_val_split:]  
partial_y_train = y_train[train_val_split:]

print(partial_x_train.shape)
print(partial_y_train.shape)

(116946, 41)
(116946,)


In [82]:
vocab_size = len(word_to_index)  # 어휘 사전의 크기
word_vector_dim = 4   # 단어 하나를 표현하는 임베딩 벡터의 차원 수입니다. 

In [85]:
# 1-D CNN 모델
model_1D = tf.keras.Sequential()
model_1D.add(tf.keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
model_1D.add(tf.keras.layers.Conv1D(16, 7, activation='relu'))
model_1D.add(tf.keras.layers.MaxPooling1D(5))
model_1D.add(tf.keras.layers.Conv1D(16, 7, activation='relu'))
model_1D.add(tf.keras.layers.GlobalMaxPooling1D())
model_1D.add(tf.keras.layers.Dense(8, activation='relu'))
model_1D.add(tf.keras.layers.Dense(1, activation='sigmoid'))  # 최종 출력은 긍정/부정을 나타내는 1dim 입니다.

model_1D.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
              
epochs=20  # 몇 epoch를 훈련하면 좋을지 결과를 보면서 바꾸어 봅시다. 

history_1D = model_1D.fit(partial_x_train,
                    partial_y_train,
                    epochs=epochs,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

# 테스트셋을 통한 모델 평가
results_1D = model_1D.evaluate(x_test,  y_test, verbose=2)

print(results_1D)

# 히스토리 저장
history_dict_1D = history_1D.history

Epoch 1/20


/data/.venv/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


229/229 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6642 - loss: 0.6059 - val_accuracy: 0.8356 - val_loss: 0.3693
Epoch 2/20
229/229 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8512 - loss: 0.3489 - val_accuracy: 0.8494 - val_loss: 0.3476
Epoch 3/20
229/229 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8666 - loss: 0.3180 - val_accuracy: 0.8478 - val_loss: 0.3525
Epoch 4/20
229/229 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8751 - loss: 0.3017 - val_accuracy: 0.8518 - val_loss: 0.3438
Epoch 5/20
229/229 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8794 - loss: 0.2909 - val_accuracy: 0.8513 - val_loss: 0.3454
Epoch 6/20
229/229 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8865 - loss: 0.2747 - val_accuracy: 0.8516 - val_loss: 0.3470
Epoch 7/20
229/229 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8927 - loss: 0.2678 - val_accuracy: 0.8528 - val_loss: 0.3556
Epoch 8/20
229/229 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8987 - loss: 0.2539 - val_accuracy: 0.8523 - val_

In [87]:
# global max pooling 1개만 사용하는 방법 : 전체 문장 중에서 단 하나의 가장 중요한 단어만 피처로 추출하여 그것으로 문장의 긍정/부정을 평가하는 방식

model_G = tf.keras.Sequential()
model_G.add(tf.keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
model_G.add(tf.keras.layers.GlobalMaxPooling1D())
model_G.add(tf.keras.layers.Dense(8, activation='relu'))
model_G.add(tf.keras.layers.Dense(1, activation='sigmoid'))  # 최종 출력은 긍정/부정을 나타내는 1dim 입니다.

model_G.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
              
epochs=20  # 몇 epoch를 훈련하면 좋을지 결과를 보면서 바꾸어 봅시다. 

history_G = model_G.fit(partial_x_train,
                    partial_y_train,
                    epochs=epochs,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

# 테스트셋을 통한 모델 평가
results_G = model_G.evaluate(x_test,  y_test, verbose=2)

print(results_G)

# 히스토리 저장
history_dict_G = history_G.history

Epoch 1/20


/data/.venv/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


229/229 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6238 - loss: 0.6780 - val_accuracy: 0.7761 - val_loss: 0.5423
Epoch 2/20
229/229 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7939 - loss: 0.4960 - val_accuracy: 0.8138 - val_loss: 0.4158
Epoch 3/20
229/229 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8312 - loss: 0.3945 - val_accuracy: 0.8251 - val_loss: 0.3890
Epoch 4/20
229/229 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8472 - loss: 0.3573 - val_accuracy: 0.8307 - val_loss: 0.3804
Epoch 5/20
229/229 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8557 - loss: 0.3395 - val_accuracy: 0.8331 - val_loss: 0.3774
Epoch 6/20
229/229 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8630 - loss: 0.3259 - val_accuracy: 0.8337 - val_loss: 0.3778
Epoch 7/20
229/229 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8683 - loss: 0.3144 - val_accuracy: 0.8331 - val_loss: 0.3795
Epoch 8/20
229/229 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8721 - loss: 0.3055 - val_accuracy: 0.8323 - val_

In [86]:
# LSTM 모델
model_L = tf.keras.Sequential()
model_L.add(tf.keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
model_L.add(tf.keras.layers.LSTM(8))   # 가장 널리 쓰이는 RNN인 LSTM 레이어를 사용하였습니다. 이때 LSTM state 벡터의 차원수는 8로 하였습니다. (변경 가능)
model_L.add(tf.keras.layers.Dense(8, activation='relu'))
model_L.add(tf.keras.layers.Dense(1, activation='sigmoid'))  # 최종 출력은 긍정/부정을 나타내는 1dim 입니다.

model_L.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
              
epochs=20  # 몇 epoch를 훈련하면 좋을지 결과를 보면서 바꾸어 봅시다. 

history_L = model_L.fit(partial_x_train,
                    partial_y_train,
                    epochs=epochs,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

# 테스트셋을 통한 모델 평가
results_L = model_L.evaluate(x_test,  y_test, verbose=2)

print(results_L)

# 히스토리 저장
history_dict_L = history_L.history

Epoch 1/20


/data/.venv/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


229/229 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.6287 - loss: 0.6383 - val_accuracy: 0.8356 - val_loss: 0.4022
Epoch 2/20
229/229 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.8363 - loss: 0.3809 - val_accuracy: 0.8442 - val_loss: 0.3566
Epoch 3/20
229/229 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.8577 - loss: 0.3373 - val_accuracy: 0.8500 - val_loss: 0.3462
Epoch 4/20
229/229 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.8660 - loss: 0.3230 - val_accuracy: 0.8496 - val_loss: 0.3480
Epoch 5/20
229/229 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.8720 - loss: 0.3097 - val_accuracy: 0.8520 - val_loss: 0.3456
Epoch 6/20
229/229 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.8731 - loss: 0.3067 - val_accuracy: 0.8511 - val_loss: 0.3467
Epoch 7/20
229/229 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.8777 - loss: 0.2998 - val_accuracy: 0.8500 - val_loss: 0.3491
Epoch 8/20
229/229 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.8779 - loss: 0.2960 - val_accuracy: 0.846

In [88]:
# 3개 모델 결과 비교 : LSTM 모델이 가장 높은 정확도를 보여줌
print(f"1D CNN model : {results_1D}")
print(f"Global MaxPooling model : {results_G}")
print(f"LSTM model : {results_L}")


1D CNN model : [0.5042845606803894, 0.8251520395278931]
Global MaxPooling model : [0.4264492690563202, 0.8257623314857483]
LSTM model : [0.3924659192562103, 0.8409992456436157]


### gensim을 활용하여 자체학습된 혹은 사전학습된 임베딩 레이어를 분석하였다.	gensim의 유사단어 찾기를 활용하여 자체학습한 임베딩과 사전학습 임베딩을 비교 분석함

In [93]:
embedding_layer = model_L.layers[0]
weights = embedding_layer.get_weights()[0]
print(weights.shape)    # shape: (vocab_size, embedding_dim)

(10000, 4)


In [95]:
# 학습한 Embedding 파라미터를 파일에 써서 저장합니다. 
word2vec_file_path = './data/word2vec.txt'
f = open(word2vec_file_path, 'w')
f.write('{} {}\n'.format(vocab_size-4, word_vector_dim))  # 몇개의 벡터를 얼마 사이즈로 기재할지 타이틀을 씁니다.

# 단어 개수(에서 특수문자 4개는 제외하고)만큼의 워드 벡터를 파일에 기록합니다. 
vectors = model_L.get_weights()[0]
for i in range(4,vocab_size):
    f.write('{} {}\n'.format(index_to_word[i], ' '.join(map(str, list(vectors[i, :])))))
f.close()

In [99]:
from gensim.models.keyedvectors import Word2VecKeyedVectors

word_vectors = Word2VecKeyedVectors.load_word2vec_format(word2vec_file_path, binary=False)
vector = word_vectors['별로']
vector

array([-0.31287396,  0.32103926,  0.31107217,  0.27470323], dtype=float32)

In [100]:
word_vectors.similar_by_word("별로")

[('곤란', 0.9996930956840515),
 ('뻥튀기', 0.9994043111801147),
 ('마이너', 0.9994020462036133),
 ('OOO', 0.9993159174919128),
 ('메기', 0.9993042945861816),
 ('화나', 0.9992968440055847),
 ('데려다가', 0.9992799162864685),
 ('지우', 0.999248206615448),
 ('싱겁', 0.9991772770881653),
 ('오만', 0.9991374611854553)]

In [122]:
from gensim.models.keyedvectors import Word2VecKeyedVectors

word2vec_file_path = './data/word2vec_ko.model'
word_vectors = Word2VecKeyedVectors.load(word2vec_file_path)
vector = word_vectors.wv["끝"] # 이건 왜 지정해주는거지?
vector

array([ 1.038851  , -1.1634965 ,  2.2386286 ,  0.8971453 , -2.088168  ,
       -4.1942534 , -0.02519768, -0.69813883, -0.2908033 , -1.4651134 ,
       -2.6863668 ,  0.7999443 , -0.6957915 , -0.5018775 ,  3.5025773 ,
        0.21326113, -0.6588098 , -1.1900496 ,  0.61399573,  2.1247969 ,
        0.1549463 ,  3.0840218 ,  3.1680267 , -3.9143991 ,  3.6073153 ,
        0.5315545 , -0.3511633 , -1.107248  ,  1.7767181 , -0.95981866,
        2.6072454 , -0.00697957, -3.27628   ,  1.558644  ,  0.81922376,
        1.9231955 ,  3.7026553 , -0.53824735, -0.4920138 ,  2.4657123 ,
       -2.620749  , -0.08950733,  0.61453176,  1.8866426 ,  0.257866  ,
       -2.7827141 ,  2.9129577 , -0.6313133 ,  2.0387425 , -4.222685  ,
       -2.7560544 ,  1.7903243 , -1.3080189 , -2.5531402 , -0.43849912,
       -3.1073573 , -0.29090825,  5.1817617 ,  1.9604833 , -5.1421604 ,
       -0.71431434,  4.5624633 , -0.55525815, -3.3019137 ,  1.6761336 ,
       -1.9606539 , -0.2726863 ,  0.2465995 ,  1.2685091 ,  1.60

In [132]:
similar_words = word_vectors.wv.most_similar(positive=['별로'], topn=10)
print(similar_words)

[('전혀', 0.6421527862548828), ('그다지', 0.6234328746795654), ('꽤', 0.6035930514335632), ('확실히', 0.5738269090652466), ('딱히', 0.5515185594558716), ('상당히', 0.542231023311615), ('볼일', 0.5412046313285828), ('다름', 0.531372606754303), ('별', 0.5228564739227295), ('확연히', 0.5193501710891724)]


In [130]:
print(word_vectors)
print(word_vectors.vocab)

Word2Vec<vocab=302960, vector_size=100, alpha=0.025>


AttributeError: 'Word2Vec' object has no attribute 'vocab'

In [127]:
embedding_matrix = np.random.rand(vocab_size, word_vector_dim)

# embedding_matrix에 Word2Vec 워드 벡터를 단어 하나씩마다 차례차례 카피한다.
for i in range(4, vocab_size):
    word = index_to_word[i]
    if word in word_vectors.wv.vocab:
        embedding_matrix[i] = word_vectors.wv.get_vector(word)

AttributeError: The vocab attribute was removed from KeyedVector in Gensim 4.0.0.
Use KeyedVector's .key_to_index dict, .index_to_key list, and methods .get_vecattr(key, attr) and .set_vecattr(key, attr, new_val) instead.
See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4

In [123]:
embedding_matrix = np.random.rand(vocab_size, word_vector_dim)

# embedding_matrix에 Word2Vec 워드 벡터를 단어 하나씩마다 차례차례 카피한다.
for i in range(4,vocab_size):
    if index_to_word[i] in word_vectors:
        embedding_matrix[i] = word_vectors[index_to_word[i]]

TypeError: argument of type 'Word2Vec' is not iterable

In [ ]:
vector = word_vectors.wv[‘끝’]

from gensim.models import KeyedVectors

word2vec = KeyedVectors.load_word2vec_format(word2vec_path, binary=True, limit=1000000)
vector = word2vec['computer']
vector     # 무려 300dim의 워드 벡터입니다.

In [91]:
from gensim.models import KeyedVectors

word2vec_path = './data/GoogleNews-vectors-negative300.bin.gz'
word2vec = KeyedVectors.load_word2vec_format(word2vec_path, binary=True, limit=1000000)
vector = word2vec['computer']
vector     # 무려 300dim의 워드 벡터입니다.

array([ 1.07421875e-01, -2.01171875e-01,  1.23046875e-01,  2.11914062e-01,
       -9.13085938e-02,  2.16796875e-01, -1.31835938e-01,  8.30078125e-02,
        2.02148438e-01,  4.78515625e-02,  3.66210938e-02, -2.45361328e-02,
        2.39257812e-02, -1.60156250e-01, -2.61230469e-02,  9.71679688e-02,
       -6.34765625e-02,  1.84570312e-01,  1.70898438e-01, -1.63085938e-01,
       -1.09375000e-01,  1.49414062e-01, -4.65393066e-04,  9.61914062e-02,
        1.68945312e-01,  2.60925293e-03,  8.93554688e-02,  6.49414062e-02,
        3.56445312e-02, -6.93359375e-02, -1.46484375e-01, -1.21093750e-01,
       -2.27539062e-01,  2.45361328e-02, -1.24511719e-01, -3.18359375e-01,
       -2.20703125e-01,  1.30859375e-01,  3.66210938e-02, -3.63769531e-02,
       -1.13281250e-01,  1.95312500e-01,  9.76562500e-02,  1.26953125e-01,
        6.59179688e-02,  6.93359375e-02,  1.02539062e-02,  1.75781250e-01,
       -1.68945312e-01,  1.21307373e-03, -2.98828125e-01, -1.15234375e-01,
        5.66406250e-02, -